In [ ]:
! pip install -q pyspark==3.2.0 spark-nlp==3.4.2

     |████████████████████████████████| 281.3 MB 31 kB/s 
     |████████████████████████████████| 142 kB 15.7 MB/s 
     |████████████████████████████████| 198 kB 56.6 MB/s 


In [ ]:
import os

In [ ]:
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
# ! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.7.5

import sparknlp

spark = sparknlp.start(spark23=True)

     |████████████████████████████████| 139 kB 29.1 MB/s 


In [ ]:
from pyspark.sql.session import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as f
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, when
from pyspark.sql.functions import udf
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, Word2Vec
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier

In [ ]:
import sparknlp

spark = sparknlp.start(gpu = True, spark23=True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os
spark

In [ ]:
spark = SparkSession.builder.appName("NLP").config("spark.executor.memory","8g").config("spark.driver.memory", "8g").getOrCreate()

In [ ]:
! wget -q https://raw.githubusercontent.com/savan77/EmotionDetectionBERT/master/nlp_train.csv
! wget -q https://raw.githubusercontent.com/savan77/EmotionDetectionBERT/master/nlp_valid.csv
! wget -q https://raw.githubusercontent.com/savan77/EmotionDetectionBERT/master/nlp_test.csv

In [ ]:
!ls -lt

total 226764
-rw-r--r--  1 root       root         1089100 Jul 16 13:49 nlp_test.csv
-rw-r--r--  1 root       root          528252 Jul 16 13:49 nlp_valid.csv
-rw-r--r--  1 root       root         4445583 Jul 16 13:49 nlp_train.csv
drwxr-xr-x  1 root       root            4096 Jul 13 13:43 sample_data
-rw-r--r--  1 root       root       226128401 Feb 22  2018 spark-2.3.0-bin-hadoop2.7.tgz
drwxr-xr-x 13 1311767953 1876110778      4096 Feb 22  2018 spark-2.3.0-bin-hadoop2.7


In [ ]:
import pandas as pd

In [ ]:
train_data = pd.read_csv('/content/nlp_train.csv')
test_data =  pd.read_csv('/content/nlp_test.csv')
valid_data =  pd.read_csv('/content/nlp_valid.csv')

In [ ]:
 train_data = pd.concat([train_data, valid_data], axis=0)

In [ ]:
# Drop N/A
train_sentiment_data = train_data.dropna()
test_sentiment_data = test_data.dropna()

In [ ]:
#[toxic, obscene]
labels = list(train_data.columns[2:])
def convert2label(row):
  concat_label = ''
  for i in range(len(row)):
    if str(row[i]) in ['1', '1.0']:
      concat_label = concat_label + labels[i] + ', '
  concat_label = concat_label[:-2] + ''
  return concat_label

In [ ]:
train_sentiment_data['labels'] = train_sentiment_data[labels].apply(lambda row: convert2label(row), axis = 1)
test_sentiment_data['labels'] = test_sentiment_data[labels].apply(lambda row: convert2label(row), axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
for df in [train_sentiment_data, test_sentiment_data]:
  df['labels'] = df[labels].apply(lambda row: convert2label(row), axis = 1)

df_train_final = train_sentiment_data[['text', 'labels']]
df_test_final = test_sentiment_data[['text', 'labels']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Convert Pandas to PySpark DataFrame, train_sentiment_data
Train = spark.createDataFrame(df_train_final)
# Convert Pandas to PySpark DataFrame, test_sentiment_data
Test = spark.createDataFrame(df_test_final) 

In [ ]:
from pyspark.sql.functions import split, col
Train = Train.withColumn('labels', split(col("labels"),","))
#
from pyspark.sql.functions import split, col
Test = Test.withColumn('labels', split(col("labels"),","))

In [ ]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")\
      .setCleanupMode("shrink")

bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset
multiClassifier = MultiClassifierDLApproach()\
      .setInputCols("sentence_embeddings")\
      .setOutputCol("category")\
      .setLabelColumn("labels")\
      .setBatchSize(64)\
      .setMaxEpochs(100)\
      .setLr(1e-3)\
      .setThreshold(0.5)\
      .setShufflePerEpoch(True)\
      .setEnableOutputLogs(True)\
      .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        bert_sent,
        multiClassifier
    ])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [ ]:
! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory


In [ ]:
%%time
pipelineModel = pipeline.fit(Train)

CPU times: user 2.59 s, sys: 307 ms, total: 2.89 s
Wall time: 8min 49s


In [ ]:
preds = pipelineModel.transform(Test)

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
preds_df = preds.select('labels','text',"category.result").toPandas()

In [ ]:
preds_df

,labels,text,result
0,"[anticipation, fear, optimism, sadness]",&gt; # First passengers exit quarantined Japan...,"[ fear, pessimism, anticipation]"
1,[neutral],"This is the best tl;dr I could make, [original...",[]
2,[disgust],Absolutely. Trade is good for everyone.\n\nBut...,"[ pessimism, anger, disgust]"
3,"[fear, pessimism]",He says the elderly should stay home. I have t...,"[ sadness, fear]"
4,"[anticipation, fear, pessimism, sadness, s...",&gt; A couple of million wont die in the US.\n...,"[ fear, optimism, anticipation]"
...,...,...,...
369,[joy],**2015 White Sox–Orioles crowdless game**\n\nO...,"[ sadness, fear, anticipation]"
370,"[anger, disgust, pessimism]",I think what the West need to understand is th...,"[ pessimism, anger, disgust]"
371,[neutral],December 31: China reports the discovery of th...,[neutral]
372,[neutral],From DeepL (Translated with www.DeepL.com/Tran...,[neutral]


In [ ]:
Prediction = spark.createDataFrame(preds_df)
Prediction

DataFrame[labels: array<string>, text: string, result: array<string>]

In [ ]:
test_preds = test_data.copy()
for label in labels:
  test_preds[label] = 0
test_preds['result'] = preds_df['result']
test_preds.head(5)

,labels,text,result,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,"[anticipation, fear, optimism, sadness]",&gt; # First passengers exit quarantined Japan...,"[ fear, pessimism, anticipation]",0,0,0,0,0,0,0,0,0,0,0,0
1,[neutral],"This is the best tl;dr I could make, [original...",[],0,0,0,0,0,0,0,0,0,0,0,0
2,[disgust],Absolutely. Trade is good for everyone.\n\nBut...,"[ pessimism, anger, disgust]",0,0,0,0,0,0,0,0,0,0,0,0
3,"[fear, pessimism]",He says the elderly should stay home. I have t...,"[ sadness, fear]",0,0,0,0,0,0,0,0,0,0,0,0
4,"[anticipation, fear, pessimism, sadness, s...",&gt; A couple of million wont die in the US.\n...,"[ fear, optimism, anticipation]",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#[toxic, obscene]
labels = list(train_data.columns[2:])
def labels2index(row):
  results = row['result']
  # print(str(results))
  results = [result.replace(' ', '') for result in results]
  for label in results:
    if label != '':
      row[label] = 1
  return row[:-1]

In [ ]:
test = test_preds.apply(lambda row: labels2index(row), axis=1)
test

,labels,text,result,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,"[anticipation, fear, optimism, sadness]",&gt; # First passengers exit quarantined Japan...,"[ fear, pessimism, anticipation]",0,1,0,1,0,0,0,1,0,0,0
1,[neutral],"This is the best tl;dr I could make, [original...",[],0,0,0,0,0,0,0,0,0,0,0
2,[disgust],Absolutely. Trade is good for everyone.\n\nBut...,"[ pessimism, anger, disgust]",1,0,1,0,0,0,0,1,0,0,0
3,"[fear, pessimism]",He says the elderly should stay home. I have t...,"[ sadness, fear]",0,0,0,1,0,0,0,0,1,0,0
4,"[anticipation, fear, pessimism, sadness, s...",&gt; A couple of million wont die in the US.\n...,"[ fear, optimism, anticipation]",0,1,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,[joy],**2015 White Sox–Orioles crowdless game**\n\nO...,"[ sadness, fear, anticipation]",0,1,0,1,0,0,0,0,1,0,0
370,"[anger, disgust, pessimism]",I think what the West need to understand is th...,"[ pessimism, anger, disgust]",1,0,1,0,0,0,0,1,0,0,0
371,[neutral],December 31: China reports the discovery of th...,[neutral],0,0,0,0,0,0,0,0,0,0,0
372,[neutral],From DeepL (Translated with www.DeepL.com/Tran...,[neutral],0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 2:], test.iloc[:, 2:], target_names=labels))

# **Prediction crawl data from comment Youtube by TrainedModel**


In [ ]:
crawl_data = (spark.read.csv(path = "results.csv",header = True,inferSchema = True).select("text"))

In [ ]:
user_regex = r"(@\w{1,15})"
hashtag_replace_regex = "#(\w{1,})"
url_regex = r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)"
email_regex = r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"
i_regex = r"i "

def cleaning_process(data):
            
    data=(data.withColumn("text",f.regexp_replace(f.col("text"), user_regex, "")) 
            .withColumn("text",f.regexp_replace(f.col("text"), hashtag_replace_regex, "$1"))            
            .withColumn("text",f.regexp_replace(f.col("text"), url_regex, "")) 
            .withColumn("text",f.regexp_replace(f.col("text"), email_regex, ""))
            .withColumn("text",f.regexp_replace(f.col("text"), "[^a-zA-Z]", " "))
            .withColumn("text",f.regexp_replace(f.col("text"), " +", " "))
            .withColumn("text",f.trim(f.col("text")))
            .withColumn("text",f.lower(f.col("text")))
            .filter(f.col("text") != ""))
    return data

In [ ]:
cleaning_process_train = cleaning_process(crawl_data)

In [ ]:
predic_crawldata = pipelineModel.transform(cleaning_process_train)

In [ ]:
preds_crawldata = predic_crawldata.select('text',"category.result").toPandas()
preds_crawldata

,text,result
0,china needs to move on from it s zero toleranc...,[]
1,babe wake up new variant dropped,"[ joy, pessimism, anticipation]"
2,excuse prevention covid pandemic no respect pe...,[ sadness]
3,it s strictly zore covid measure for all peopl...,[disgust]
4,africans we where slaves years ago,[joy]
...,...,...
13796,carlette southern robert,"[joy, neutral, anticipation]"
13797,i m vaxed and do not feel foolish at all it ha...,"[ sadness, fear, joy, optimism, pessimism,..."
13798,ignorance is bliss,[disgust]
13799,just got my second booster last week,"[joy, anticipation]"


In [ ]:
preds_dataframe = spark.createDataFrame(preds_crawldata)

In [ ]:
preds_crawldata.to_csv("pred_crawl.csv", index = False)

In [ ]:
test_preds_you = preds_crawldata.copy()
for label in labels:
  test_preds_you[label] = 0
test_preds_you['result'] = preds_crawldata['result']
test_preds_you.head(5)

,text,result,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,china needs to move on from it s zero toleranc...,[],0,0,0,0,0,0,0,0,0,0,0,0
1,babe wake up new variant dropped,"[ joy, pessimism, anticipation]",0,0,0,0,0,0,0,0,0,0,0,0
2,excuse prevention covid pandemic no respect pe...,[ sadness],0,0,0,0,0,0,0,0,0,0,0,0
3,it s strictly zore covid measure for all peopl...,[disgust],0,0,0,0,0,0,0,0,0,0,0,0
4,africans we where slaves years ago,[joy],0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
crawl_you = test_preds_you.apply(lambda row: labels2index(row), axis=1)
crawl_you

,text,result,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,china needs to move on from it s zero toleranc...,[],0,0,0,0,0,0,0,0,0,0,0
1,babe wake up new variant dropped,"[ joy, pessimism, anticipation]",0,1,0,0,1,0,0,1,0,0,0
2,excuse prevention covid pandemic no respect pe...,[ sadness],0,0,0,0,0,0,0,0,1,0,0
3,it s strictly zore covid measure for all peopl...,[disgust],0,0,1,0,0,0,0,0,0,0,0
4,africans we where slaves years ago,[joy],0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13796,carlette southern robert,"[joy, neutral, anticipation]",0,1,0,0,1,0,0,0,0,0,0
13797,i m vaxed and do not feel foolish at all it ha...,"[ sadness, fear, joy, optimism, pessimism,...",0,1,0,1,1,1,1,1,1,0,0
13798,ignorance is bliss,[disgust],0,0,1,0,0,0,0,0,0,0,0
13799,just got my second booster last week,"[joy, anticipation]",0,1,0,0,1,0,0,0,0,0,0


In [ ]:
crawl_you.to_csv("crawl_you.csv", index = False)

In [ ]:
preds_dataframe.select('*').limit(20).show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|china needs to mo...|                  []|
|babe wake up new ...|[ joy,  pessimism...|
|excuse prevention...|          [ sadness]|
|it s strictly zor...|           [disgust]|
|africans we where...|               [joy]|
|         sigrid kaag|     [joy, optimism]|
|zero covid s sort...|[ pessimism, ange...|
|this zero covid p...|        [ pessimism]|
|        so primitive|                  []|
|free hong kong do...|                  []|
|               karma|[ sadness, joy,  ...|
|always imitated n...|[ sadness,  joy, ...|
|its dum to take o...|                  []|
|zero common sense...|        [ pessimism]|
|they still don t ...|[ sadness, antici...|
|lockdown forever ...|[ fear,  pessimis...|
|as long as we bre...|[joy, neutral, an...|
|again when this w...|[ sadness,  fear,...|
|hope they can fin...|[joy,  optimism, ...|
|i just don t thin...|[ sadness,

In [ ]:
#END